In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('res.csv', sep=',')
df['num_qubits'] = df['num_qubits'].astype(int)
df.replace(to_replace=["None"], value=0, inplace=True)
df.set_index('num_qubits', inplace=True)
df

In [ ]:
df['time_new_scheme_with_opt'] = df['time_new_scheme_with_opt'].astype(float)
df['time_baseline_O0'] = df['time_baseline_O0'].astype(float)
df['time_baseline_O3'] = df['time_baseline_O3'].astype(float)
df['time_new_scheme_without_opt'] = df['time_new_scheme_without_opt'].astype(float)
df['cx_count_with_opt'] = df['cx_count_with_opt'].astype(int)
df['cx_count_without_opt'] = df['cx_count_without_opt'].astype(int)
df['cx_count_baseline_O0'] = df['cx_count_baseline_O0'].astype(int)
df['cx_count_baseline_O3'] = df['cx_count_baseline_O3'].astype(int)

df["time_ratio_with_opt_O0"] = df["time_new_scheme_with_opt"]/df["time_baseline_O0"]
df["time_ratio_with_opt_O3"] = df["time_new_scheme_with_opt"]/df["time_baseline_O3"]
#df["time_ratio_without_opt"] = df["time_new_scheme_without_opt"]/df["time_baseline"]
df["cx_ratio_with_opt_O0"] = df['cx_count_with_opt'].divide(df['cx_count_baseline_O0'])
df["cx_ratio_with_opt_O3"] = df['cx_count_with_opt'].divide(df['cx_count_baseline_O3'])
#df["cx_ratio_without_opt"] = df['cx_count_without_opt'].divide(df['cx_count_baseline'])
df

In [ ]:
#df.groupby(['considered_following_qubits', 'sample_probability'])['cx_ratio_with_opt'].plot(legend=True);

In [ ]:
#df.groupby(['considered_following_qubits', 'sample_probability'])['time_ratio_with_opt'].plot(legend=True);

In [ ]:
x = df[df.considered_following_qubits==1000]
x = x[x.index%5==0][[
    "sample_probability",
    "time_baseline_O0",
    "time_baseline_O3",
    "time_new_scheme_with_opt",
    "time_ratio_with_opt_O0", 
    "time_ratio_with_opt_O3", 
    "cx_count_baseline_O0", 
    "cx_count_baseline_O3", 
    "cx_count_with_opt",
    "cx_ratio_with_opt_O0",
    "cx_ratio_with_opt_O3",
]
]

x["time_baseline_O0"] = x["time_baseline_O0"].round(2)
x["time_baseline_O3"] = x["time_baseline_O3"].round(2)
x["time_new_scheme_with_opt"] = x["time_new_scheme_with_opt"].round(2)
x["cx_ratio_with_opt_O0"] = x["cx_ratio_with_opt_O0"].round(2)
x["cx_ratio_with_opt_O3"] = x["cx_ratio_with_opt_O3"].round(2)
x["time_ratio_with_opt_O0"] = x["time_ratio_with_opt_O0"].round(2)
x["time_ratio_with_opt_O3"] = x["time_ratio_with_opt_O3"].round(2)
x

In [ ]:
x["Benchmark"] = "Qubits=" + x.index.astype(str) + ", p=" + x.sample_probability.astype(str)

In [ ]:
x = x.sort_values(["sample_probability", "num_qubits"])
x.set_index('Benchmark', inplace=True)
x = x.drop('sample_probability', axis=1)
x

In [ ]:
x = x.rename(columns={"time_ratio_with_opt_O0": "Time Ratio O0", 
                      "time_ratio_with_opt_O3": "Time Ratio O3", 
                      "cx_count_baseline_O0": "CX Count Baseline O0",
                      "cx_count_baseline_O3": "CX Count Baseline O3", 
                      "cx_count_with_opt": "CX Count New Scheme", 
                      "cx_ratio_with_opt_O0": "CX Count Ratio O0", 
                      "cx_ratio_with_opt_O3": "CX Count Ratio O3", 
                      "time_new_scheme_with_opt":"Time New Scheme", 
                      "time_baseline_O0":"Time Baseline O0",
                      "time_baseline_O3":"Time Baseline O3"
                     }
            )

In [ ]:
print(x.to_latex())